# Data Pre-Processing

## Location Data

**Needed Fields:**
- Postcode : int
- Locality : str (suburb)
- State : str
- Long : float
- Lat : float

In [52]:
import pandas as pd

postcode_df = pd.read_excel("Datasets/australian_postcodes.xlsx")

postcode_df = postcode_df[['Postcode', 'Locality', 'State', 'Long', 'Lat']]

postcode_df

,Postcode,Locality,State,Long,Lat
0,200,ANU,ACT,149.119000,-35.277700
1,200,Australian National University,ACT,149.118900,-35.277700
2,800,DARWIN,NT,130.836680,-12.458684
3,800,DARWIN CITY,NT,130.836680,-12.458684
4,801,DARWIN,NT,130.836680,-12.458684
...,...,...,...,...,...
18521,9013,BRISBANE,QLD,152.823141,-27.603479
18522,9015,BRISBANE,QLD,152.823141,-27.603479
18523,9464,NORTHGATE MC,QLD,153.074982,-27.397055
18524,9726,GOLD COAST MC,QLD,153.412197,-28.008783


In [53]:

postcode_filter = postcode_df.loc[:, ['Postcode', 'Locality', 'State', 'Long', 'Lat']].copy()
postcode_filter = postcode_filter.drop_duplicates(subset=['Postcode', 'Long', 'Lat'], keep="first")

postcode_filter.loc[:, 'Postcode'] = postcode_filter['Postcode'].astype(str)
postcode_filter.loc[:, 'Locality'] = postcode_filter['Locality'].astype(str)
postcode_filter.loc[:, 'State'] = postcode_filter['State'].astype(str)

postcode_filter['Postcode'] = postcode_filter['Postcode'].str.zfill(4)
postcode_filter['Locality'] = postcode_filter['Locality'].str.upper()
postcode_filter['State'] = postcode_filter['State'].str.upper()

postcode_filter.head(5)

,Postcode,Locality,State,Long,Lat
0,0200,ANU,ACT,149.119000,-35.277700
1,0200,AUSTRALIAN NATIONAL UNIVERSITY,ACT,149.118900,-35.277700
2,0800,DARWIN,NT,130.836680,-12.458684
4,0801,DARWIN,NT,130.836680,-12.458684
5,0803,WAGAIT BEACH,NT,130.745908,-12.433991


In [54]:
suburbs = postcode_filter['Locality'].drop_duplicates().reset_index()
suburbs['locality_id'] = ['SUB{:02d}'.format(i+1) for i in range(len(suburbs))]

suburbs = suburbs[['locality_id', 'Locality']]

suburbs

,locality_id,Locality
0,SUB01,ANU
1,SUB02,AUSTRALIAN NATIONAL UNIVERSITY
2,SUB03,DARWIN
3,SUB04,WAGAIT BEACH
4,SUB05,PARAP
...,...,...
4564,SUB4565,LITTLE LONSDALE STREET
4565,SUB4566,DANDENONG
4566,SUB4567,NORTHGATE MC
4567,SUB4568,GOLD COAST MC


In [55]:
postcode_filter['Locality'] = postcode_filter['Locality'].astype(str)
suburbs['Locality'] = suburbs['Locality'].astype(str)

postcode_filter = postcode_filter.join(suburbs.set_index('Locality'), on='Locality')

postcode_filter.rename(columns={'Postcode': 'postcode',
                        'Locality': 'locality',
                        'State': 'state_code',
                        'Long': 'long',
                        'Lat': 'lat'},
                        inplace=True)

postcode_filter

,postcode,locality,state_code,long,lat,locality_id
0,0200,ANU,ACT,149.119000,-35.277700,SUB01
1,0200,AUSTRALIAN NATIONAL UNIVERSITY,ACT,149.118900,-35.277700,SUB02
2,0800,DARWIN,NT,130.836680,-12.458684,SUB03
4,0801,DARWIN,NT,130.836680,-12.458684,SUB03
5,0803,WAGAIT BEACH,NT,130.745908,-12.433991,SUB04
...,...,...,...,...,...,...
18521,9013,BRISBANE,QLD,152.823141,-27.603479,SUB2386
18522,9015,BRISBANE,QLD,152.823141,-27.603479,SUB2386
18523,9464,NORTHGATE MC,QLD,153.074982,-27.397055,SUB4567
18524,9726,GOLD COAST MC,QLD,153.412197,-28.008783,SUB4568


In [ ]:
postcode_filter['locality_id'] = ['LOC{:02d}'.format(i+1) for i in range(len(postcode_filter))]

,postcode,locality,state_code,long,lat,locality_id
0,0200,ANU,ACT,149.119000,-35.277700,LOC01
1,0200,AUSTRALIAN NATIONAL UNIVERSITY,ACT,149.118900,-35.277700,LOC02
2,0800,DARWIN,NT,130.836680,-12.458684,LOC03
4,0801,DARWIN,NT,130.836680,-12.458684,LOC04
5,0803,WAGAIT BEACH,NT,130.745908,-12.433991,LOC05
...,...,...,...,...,...,...
18521,9013,BRISBANE,QLD,152.823141,-27.603479,LOC5124
18522,9015,BRISBANE,QLD,152.823141,-27.603479,LOC5125
18523,9464,NORTHGATE MC,QLD,153.074982,-27.397055,LOC5126
18524,9726,GOLD COAST MC,QLD,153.412197,-28.008783,LOC5127


## State List Master Table

In [57]:
state_list = postcode_filter['state_code'].drop_duplicates().reset_index()
state_list['state_id'] = ['STATE{:02d}'.format(i+1) for i in range(len(state_list))]

state_list = state_list[['state_id', 'state_code']]

full_state = {"ACT": "AUSTRALIAN CAPITAL TERRITORY",
              "NT": "NORTHERN TERRITORY",
              "SA": "SOUTH AUSTRALIA",
              "WA": "WESTERN AUSTRALIA",
              "NSW": "NEW SOUTH WALES",
              "QLD": "QUEENSLAND",
              "VIC": "VICTORIA",
              "TAS": "TASMANIA"}

state_list['state_name'] = state_list['state_code'].map(full_state)
state_list["state_code"] = state_list["state_code"]

# state_list = state_list.drop(["state_code"], axis='columns')
state_list

,state_id,state_code,state_name
0,STATE01,ACT,AUSTRALIAN CAPITAL TERRITORY
1,STATE02,NT,NORTHERN TERRITORY
2,STATE03,SA,SOUTH AUSTRALIA
3,STATE04,WA,WESTERN AUSTRALIA
4,STATE05,NSW,NEW SOUTH WALES
5,STATE06,QLD,QUEENSLAND
6,STATE07,VIC,VICTORIA
7,STATE08,TAS,TASMANIA


In [58]:
state_list.to_csv("state_list.csv")

Assigning state_id to location data

In [60]:
postcode_filter = postcode_filter.join(state_list.set_index('state_code'), on='state_code')

postcode_filter = postcode_filter[['locality_id', 'state_id', 'postcode', 'locality',
                                   'long', 'lat']]

postcode_filter

,locality_id,state_id,postcode,locality,long,lat
0,LOC01,STATE01,0200,ANU,149.119000,-35.277700
1,LOC02,STATE01,0200,AUSTRALIAN NATIONAL UNIVERSITY,149.118900,-35.277700
2,LOC03,STATE02,0800,DARWIN,130.836680,-12.458684
4,LOC04,STATE02,0801,DARWIN,130.836680,-12.458684
5,LOC05,STATE02,0803,WAGAIT BEACH,130.745908,-12.433991
...,...,...,...,...,...,...
18521,LOC5124,STATE06,9013,BRISBANE,152.823141,-27.603479
18522,LOC5125,STATE06,9015,BRISBANE,152.823141,-27.603479
18523,LOC5126,STATE06,9464,NORTHGATE MC,153.074982,-27.397055
18524,LOC5127,STATE06,9726,GOLD COAST MC,153.412197,-28.008783


In [61]:
postcode_filter.to_csv('location_data.csv')